In [1]:
# Core
import numpy as np
import pandas as pd

# Tensorflow / ML
import tensorflow as tf

# Utility
import os

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
    
# Utility
import time

In [2]:
# Set visibible GPU
gpu_num: int = 1
os.environ['CUDA_VISIBLE_DEVICES']=f'{gpu_num}'

In [3]:
# Configure TF GPU growth
import kepler_sieve
from tf_utils import gpu_grow_memory, get_gpu_device
gpu_grow_memory(verbose=True)

Found 0 GPUs.  Setting memory growth = True.


In [4]:
# MSE Imports
from asteroid_search import load_fitted_elts, load_ztf_hits, append_fitted_elt, append_ztf_hit, file_name_model
from asteroid_element import load_ast_elt
from candidate_element import asteroid_elts, perturb_elts, random_elts, elts_add_mixture_params, elts_add_H
from random_elements import load_best_random_elts, make_ztf_ast
from ztf_ast import load_ztf_nearest_ast, calc_hit_freq
from ztf_element import load_ztf_batch, make_ztf_batch, ztf_score_by_elt, ztf_elt_summary
from asteroid_model import AsteroidPosition, AsteroidDirection, make_model_ast_pos
from asteroid_search_layers import CandidateElements, MixtureParameters, TrajectoryScore
from asteroid_search_model import AsteroidSearchModel
from asteroid_search_report import traj_diff
from nearest_asteroid import nearest_ast_elt_cart, nearest_ast_elt_cov, elt_q_norm
from element_eda import score_by_elt
from asteroid_dataframe import calc_ast_data, spline_ast_vec_df
from astro_utils import deg2dist, dist2deg, dist2sec

In [5]:
# Aliases
keras = tf.keras

# Constants
dtype = tf.float32
dtype_np = np.float32
space_dims = 3

In [6]:
# Set plot style variables
mpl.rcParams['figure.figsize'] = [16.0, 10.0]
mpl.rcParams['font.size'] = 16

## Review Fitted Elements

In [7]:
known_ast = False

In [14]:
fitted_elts = load_fitted_elts(known_ast=known_ast, min_hits=6)
fitted_elts.hits = np.round(fitted_elts.hits)
fitted_elts.sort_values(['hits', 'R_sec'], ascending=[False, True], inplace=True)
best_element_id = fitted_elts.element_id.iloc[0]

In [15]:
fitted_elts

,element_id,a,e,inc,Omega,omega,f,epoch,num_hits,R_sec,thresh_sec,log_like,hits,num_rows_close,timestamp
element_id,,,,,,,,,,,,,,,
44117,44117,2.935864,0.187419,0.124515,3.166528,1.230836,-3.122138,58600.0,9.990374,13.031281,863.441772,57.351826,10.0,29.0,2020-04-24 20:09:21.720375
3308,3308,3.026904,0.119945,0.129432,3.903342,4.525802,4.818909,58600.0,9.921326,5.803865,336.060303,59.502411,9.0,14.0,2020-04-23 22:58:22.891494
45801,45801,2.754677,0.047293,0.118126,3.139070,5.767782,-1.949476,58600.0,8.976903,12.320272,589.068054,50.120068,9.0,18.0,2020-04-24 20:53:59.991079
113970,113970,2.897023,0.068932,0.209250,5.663728,3.868474,4.450756,58600.0,7.983499,14.468097,690.245483,46.563499,8.0,14.0,2020-04-26 10:51:54.459207
96507,96507,2.820178,0.069025,0.080228,2.221972,0.960994,-2.630978,58600.0,9.045378,18.369638,750.331299,42.609283,8.0,28.0,2020-04-26 01:52:35.212885
50775,50775,2.374712,0.100280,0.165483,4.280114,5.955170,2.995417,58600.0,9.970323,19.354061,862.389160,47.708138,8.0,31.0,2020-04-24 23:35:22.881139
83437,83437,2.225388,0.178568,0.199073,4.389083,0.745947,1.734980,58600.0,10.886101,25.667967,943.090210,43.075562,8.0,44.0,2020-04-25 19:15:13.291287
107397,107397,2.495918,0.012984,0.242359,2.332142,3.275486,1.210875,58600.0,25.143215,94.019241,1903.446167,64.290276,8.0,103.0,2020-04-26 07:35:10.998538
73437,73437,2.851923,0.156586,0.173418,1.826442,3.073785,-3.750503,58600.0,7.989920,10.397193,372.861481,39.857136,7.0,14.0,2020-04-25 14:09:11.584352


In [16]:
fitted_elts.shape[0]

38

In [17]:
fitted_elts.loc[best_element_id]

element_id                             44117
a                                    2.93586
e                                   0.187419
inc                                 0.124515
Omega                                3.16653
omega                                1.23084
f                                   -3.12214
epoch                                  58600
num_hits                             9.99037
R_sec                                13.0313
thresh_sec                           863.442
log_like                             57.3518
hits                                      10
num_rows_close                            29
timestamp         2020-04-24 20:09:21.720375
Name: 44117, dtype: object

In [18]:
cols_elt = ['a', 'e', 'inc', 'Omega', 'omega', 'f', 'epoch']
best_elt = fitted_elts[cols_elt].loc[best_element_id]

In [19]:
# fitted_elts.to_csv('../data/candidate_elt/fitted_elts_best.csv')

## Review ZTF Hits

In [20]:
ztf_hits = load_ztf_hits(known_ast=known_ast, display=True, min_hits=3)

In [21]:
ztf_hits.loc[best_element_id]

,element_id,ztf_id,ObjectID,CandidateID,mjd,ra,dec,mag_app,ux,uy,uz,elt_ux,elt_uy,elt_uz,s_sec,timestamp
ztf_id,,,,,,,,,,,,,,,,
166353,44117,166353,b'ZTF17aadmibo',780127811615015006,58534.127813,47.009013,11.179460,17.146601,0.668944,0.735490,-0.107554,0.668945,0.735490,-0.107548,1.090814,2020-04-24 20:09:21.720375
166354,44117,166354,b'ZTF17aadmibo',780127342915015004,58534.127350,47.009206,11.179696,17.139999,0.668941,0.735493,-0.107550,0.668946,0.735489,-0.107548,1.366716,2020-04-24 20:09:21.720375
3594638,44117,3594638,b'ZTF19acvrose',1058437713515015041,58812.437720,116.913738,12.251436,19.568300,-0.442340,0.883886,-0.151924,-0.442339,0.883883,-0.151945,4.505251,2020-04-24 20:09:21.720375
5466244,44117,5466244,b'ZTF18aaaweau',1145189806215010006,58899.189803,103.133095,14.445428,17.902100,-0.220031,0.964467,-0.146253,-0.220054,0.964460,-0.146262,5.217349,2020-04-24 20:09:21.720375
5466245,44117,5466245,b'ZTF18aaaweau',1145189806215010007,58899.189803,103.133104,14.445456,17.550800,-0.220031,0.964467,-0.146253,-0.220054,0.964460,-0.146262,5.234842,2020-04-24 20:09:21.720375
5466246,44117,5466246,b'ZTF18aaaweau',1145189806215010008,58899.189803,103.133093,14.445428,17.789000,-0.220031,0.964467,-0.146253,-0.220054,0.964460,-0.146262,5.222844,2020-04-24 20:09:21.720375
5467623,44117,5467623,b'ZTF18aaaweau',1145190705115010005,58899.190706,103.133005,14.445566,17.205000,-0.220029,0.964468,-0.146251,-0.220052,0.964461,-0.146261,5.410742,2020-04-24 20:09:21.720375
5475727,44117,5475727,b'ZTF18aaaweau',1145208286215010004,58899.208287,103.133007,14.445438,17.782801,-0.220029,0.964467,-0.146253,-0.220027,0.964468,-0.146248,1.223236,2020-04-24 20:09:21.720375
5475728,44117,5475728,b'ZTF18aaaweau',1145233466215010000,58899.233461,103.133107,14.445611,16.757299,-0.220031,0.964467,-0.146250,-0.219992,0.964480,-0.146229,9.505803,2020-04-24 20:09:21.720375


In [ ]:
cols = ['ObjectID', 'CandidateID', 'mjd', 'ra', 'dec', 'mag_app', 's_sec']
best_hits  = ztf_hits[cols].loc[best_element_id].reset_index()

In [ ]:
best_hits

In [ ]:
# best_hits.to_csv('../data/candidate_elt/ztf_hits_best.csv')

In [ ]:
np.mean(ztf_hits.s_sec.loc[best_element_id])

In [ ]:
ztf_id_close = ztf_hits.ztf_id.loc[best_element_id]

In [ ]:
# ztf_hits.to_csv('../data/candidate_elt/ztf_hits_unknown.csv')

## Load ZTF Data and Batch of Orbital Elements

In [ ]:
# Load orbital elements for known asteroids
ast_elt = load_ast_elt()

# Number of asteroids
N_ast = ast_elt.shape[0]

In [ ]:
# Parameters to build random orbital elements and ztf_ast
batch_size_init = 1024
batch_size = 64
random_seed = best_element_id // batch_size
thresh_deg = 2.0

In [ ]:
# Load ztf nearest asteroid data
ztf_ast = make_ztf_ast(known_ast=known_ast)

In [ ]:
# Review ztf_ast
ztf_ast

In [ ]:
# Best random elements
elts = load_best_random_elts(random_seed=random_seed, known_ast=known_ast, 
                             batch_size_init=batch_size_init, batch_size=batch_size, thresh_deg=thresh_deg)

In [ ]:
# # Review best random elements
# elts

In [ ]:
ztf_ast.loc[ztf_id_close]

In [ ]:
close_ast_num = ztf_ast.nearest_ast_num.loc[ztf_id_close].mode().values[0]
close_ast_num

In [ ]:
nearest_ast_dist = ztf_ast.nearest_ast_dist.loc[ztf_id_close]

In [ ]:
nearest_ast_sec = dist2sec(nearest_ast_dist).values
nearest_ast_sec

In [ ]:
np.mean(nearest_ast_sec)

In [ ]:
ast_elt.Name.loc[close_ast_num]

In [ ]:
near_elt = ast_elt[cols_elt].loc[close_ast_num]
near_elt

In [ ]:
best_elt - near_elt

## Batches of ZTF Data Near Initial Candidate Elements

In [ ]:
# Arguments to make_ztf_batch
near_ast = False
regenerate = False

In [ ]:
# Load perturbed element batch
ztf_elt = load_ztf_batch(elts=elts, ztf=ztf_ast, thresh_deg=thresh_deg, near_ast=near_ast, regenerate=regenerate)

In [ ]:
# # Review ZTF elements
# ztf_elt

In [ ]:
# Mixture parameters
num_hits: int = 10
R_deg: float = 0.5

In [ ]:
# Add mixture parameters to candidate elements
elts_add_mixture_params(elts=elts, num_hits=num_hits, R_deg=R_deg, thresh_deg=thresh_deg)

In [ ]:
# Add brightness parameter H
elts_add_H(elts=elts)

In [ ]:
# # Review perturbed elements
# elts

## Build Asteroid Search Model

In [ ]:
# Observatory for ZTF data is Palomar Mountain
site_name = 'palomar'

In [ ]:
# Training parameters
learning_rate = 2.0**-12
clipnorm = 1.0

In [ ]:
# The file name for the saved model
file_name = file_name_model(seed=random_seed, known_ast=known_ast, 
                            batch_size_init=batch_size_init, batch_size=batch_size, thresh_deg=thresh_deg)

In [ ]:
# Build asteroid search model
model = AsteroidSearchModel(
                elts=elts, ztf_elt=ztf_elt, 
                site_name=site_name, thresh_deg=thresh_deg, 
                learning_rate=learning_rate, clipnorm=clipnorm,
                name='model',
                file_name=file_name,
)

In [ ]:
# Load trained model
model.load()

In [ ]:
# Report before training starts
model.report()

In [ ]:
# Generate fitted elements
elts_fit = model.candidates_df()

In [ ]:
cols = ['element_id', 'a', 'e', 'inc', 'Omega', 'omega', 'f', 'epoch',
        'num_hits', 'R', 'R_deg', 'R_sec', 'R_max', 'R_deg_max', 'thresh_s',
        'thresh_deg', 'thresh_sec', 'log_like', 'hits', 'num_rows_close']

In [ ]:
# elts_fit[cols]

In [ ]:
# Save ztf_hits to ztf_hits_known_ast or ztf_unknown_ast

In [ ]:
# Generate ZTF hits
ztf_hit = model.calc_ztf_hits()

In [ ]:
# ztf_hit

In [ ]:
elts_fit, elts_near_ast = model.nearest_ast()

In [ ]:
# Review asteroids nearest to the fitted elements
cols = ['element_id', 'log_like', 'hits', 'R_sec', 'thresh_sec', 'nearest_ast_num', 'nearest_ast_dist', 'nearest_ast_q_norm']

# Summary
has_hits = (model.elts_fit.hits >= 5)
is_converged = has_hits
num_converged = np.sum(is_converged)

# Mean on converged
mean_hits = np.mean(model.elts_fit.hits[is_converged])
mean_R_sec = np.mean(model.elts_fit.R_sec[is_converged])
err_cart_mean = np.mean(model.elts_near_ast.nearest_ast_dist[is_converged])
err_cart_geo = np.exp(np.mean(np.log(model.elts_near_ast.nearest_ast_dist[is_converged])))
err_cov_mean = np.exp(np.mean(np.log(model.elts_near_ast.nearest_ast_q_norm[is_converged])))

# Report
print(f'Good convergence on {num_converged} asteroids.')
print(f'Mean on converged asteroids:')
print(f'Number of hits         : {mean_hits:5.2f}')
print(f'Resolution in arc sec  : {mean_R_sec:5.2f}')
print(f'GeoMean Distance in AU : {err_cart_geo:5.2e}')
print(f'Covariance norm        : {err_cov_mean:5.2e}')

In [ ]:
# Review good matches
model.elts_fit[cols][is_converged].sort_values(by=['hits', 'R_sec'], ascending=[False, True])

In [ ]:
from nearest_asteroid import calc_elt_pos, ts

In [ ]:
best_loc = best_element_id % batch_size
elts.loc[best_loc:best_loc]

In [ ]:
q_best_elt = calc_elt_pos(elts.loc[best_loc:best_loc], ts)

In [ ]:
q_near_ast = calc_elt_pos(ast_elt.loc[close_ast_num:close_ast_num], ts)

In [ ]:
mean_dist = np.mean(np.linalg.norm(q_best_elt-q_near_ast, axis=2))
mean_dist

In [ ]:
elt_q_norm(elts.loc[best_loc:best_loc], np.array([close_ast_num]))